# Create hybridization.json from directory

This only needs to be run to generate the org.json file

In [1]:
import os
import re
import json
import numpy as np
import glob
from skimage.io import imread, imsave
from collections import Counter, OrderedDict
from typing import Mapping, Dict, List

from starfish.constants import Indices
import hashlib
from itertools import product

In [2]:
def tile_index_iterator(indices: Dict[Indices, int]):
    """yield indices of tiles in the order they appear in a glob pattern (note: indices must be properly specified for this to work)
    
    Parameters
    ----------
    indices : OrderedDict[Indices, int]  # mypy won't accept ordereddict
        mapping of Indices to the size of the dimension, listed in c-order to match how the indices change on the file system (last index
        changes fastest)
    
    Yields:
    ------
    Dict[Indices, int] : 
        mapping of the indices to the value/index for a specific tile. If input indices are specified properly, this will match the 
        list of files globbed from the file system
    """
    names = []
    for k, v in indices.items():
        inds = [(k, i) for i in range(v)]
        names.append(inds)
    for tile_indices in product(*names):
        yield dict(tile_indices)

In [3]:
def file_hash(filename: str) -> str:
    """return sha256 hash for file"""
    h = hashlib.sha256()
    with open(filename, 'rb', buffering=0) as f:
        for b in iter(lambda : f.read(128*1024), b''):
            h.update(b)
    return h.hexdigest()

In [4]:
def create_hybridization_json(
    filenames: List[str], ordered_indices: Dict[Indices, int], default_tile_shape=[2048, 2048], default_tile_format='TIFF') -> dict:
    """
    Parameters
    ----------
    filenames : List[str]
        ordered list of file names, should match the iterator generated by ordered_indices
    ordered_indices : Dict[Indices, int]
        order for indices (see tile_index_iterator for more details)
    
    Returns
    -------
    dict : 
        hybridization json file in starfish v0.0.0 format
        
    """
    hashes = [file_hash(f) for f in filenames]
    ordered_tile_indices = tile_index_iterator(ordered_indices)
    tiles = []
    
    for tile_indices, hash_, file_name in zip(ordered_tile_indices, hashes, filenames):
        tiles.append(
            {
                "coordinates": {
                    "x": [0, 0.0001],
                    "y": [0, 0.0001],
                    "z": [0, 0.0001],
                },
                "indices": {k.value: v for (k, v) in tile_indices.items()},
                "file": os.path.basename(file_name),
                "sha256": hash_
            }
        )
    
    return {
        "version": "0.0.0",
        "dimensions": ["x", "y"] + list(k.value for k in ordered_indices.keys()),
        "default_tile_shape": default_tile_shape,
        "default_tile_format": default_tile_format,
        "shape": {
            f"{Indices.HYB.value}": ordered_indices[Indices.HYB],
            f"{Indices.CH.value}": ordered_indices[Indices.CH],
            f"{Indices.Z.value}": ordered_indices[Indices.Z],
        },
        "tiles": tiles
    }

In [5]:
# simple experiment metadata object
experiment_metadata = {
    "version": "0.0.0",
    "hybridization_images": "hybridization.json",
    "auxiliary_images": {
        "nuclei": "nuclei.json"
    }
}

In [6]:
# faked up codebook
simone_codebook = [
    {
        "codeword": {'c': 0, 'h': 0, 'v': 1}, 
        "gene_name": "gene_1"
    },
    {
        "codeword": {'c': 1, 'h': 0, 'v': 1},
        "gene_name": "gene_2"
    }
]

In [7]:
# this sort is needed to put globbed files into a logical order that works with the tile_index_iterator object
def alphanum_sort(list_): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(list_, key = alphanum_key)

# Create an Experiment

In [8]:
data_prefix = os.path.expanduser('~/google_drive/starfish/data/linnarsson/osmFISH/fov_00{series}')

In [11]:
# simone's data has 3 fovs, fov_001, fov_002, and fov_003
for series in range(1, 4):

    # get the files, split nuclei from genes
    all_files = glob.glob(os.path.join(data_prefix.format(series=series), '*.tif'))
    nuclei = alphanum_sort([f for f in all_files if 'c003' in f])
    genes = alphanum_sort([f for f in all_files if 'c003' not in f])

    # make the index dict for gene files
    sorted_gene_indices = OrderedDict([
        (Indices.Z, 39),
        (Indices.CH, 2),
        (Indices.HYB, 1),
    ])

    # make the index dict for nuclei files
    sorted_nuclei_indices = OrderedDict([
        (Indices.Z, 39),
        (Indices.CH, 1),
        (Indices.HYB, 1),
    ])

    # create the hybridization json
    hyb_json = create_hybridization_json(
        genes, 
        sorted_gene_indices,
        default_tile_shape=[2048, 2048],
        default_tile_format='TIFF'
    )

    # ... and the nuclei json
    nuclei_json = create_hybridization_json(
        nuclei, 
        sorted_nuclei_indices,
        default_tile_shape=[2048, 2048],
        default_tile_format='TIFF'
    )

    # write everything to disk
    with open(os.path.join(data_prefix.format(series=series), 'hybridization.json'), 'w') as f:
        json.dump(hyb_json, f)
    with open(os.path.join(data_prefix.format(series=series), 'nuclei.json'), 'w') as f:
        json.dump(nuclei_json, f)
    with open(os.path.join(data_prefix.format(series=series), 'experiment.json'), 'w') as f:
        json.dump(experiment_metadata, f)
    with open(os.path.join(data_prefix.format(series=series), 'codebook.json'), 'w') as f:
        json.dump(simone_codebook, f)

# Load the new spec into Starfish

In [12]:
from starfish.io import Stack
Stack.from_experiment_json(os.path.join(data_prefix.format(series=1), 'experiment.json'))